In [1]:
import hdf5plugin
import h5py
import numpy as np
import pandas as pd

In [2]:
filename = "train_cite_inputs.h5"

with h5py.File(filename,'r') as hdf:
    ls = list(hdf.keys())
#     print("list of datasets in this file : \n",ls)
#     print("list of keys in dataset ",hdf.get('train_cite_inputs').keys())
    
#     print(hdf['train_cite_inputs']['axis0'])
    axis0 = hdf.get('train_cite_inputs').get('axis0')
    axis0_arr = np.array(axis0)
    axis0_arr = np.char.decode(axis0_arr) 
#     print(axis0_arr) 
    
#     print(hdf['train_cite_inputs']['axis1'])
    axis1 = hdf.get('train_cite_inputs').get('axis1')
    axis1_arr = np.array(axis1)
    axis1_arr = np.char.decode(axis1_arr)
#     print(axis1_arr)
    
#     print(hdf['train_cite_inputs']['block0_items'])
    block0_items = hdf.get('train_cite_inputs').get('block0_items')
    block0_items_arr = np.array(block0_items)
#     print(block0_items_arr)
    
    
#     print(hdf['train_cite_inputs']['block0_values'])
    block0_values = hdf.get('train_cite_inputs').get('block0_values')
    block0_values_arr = np.array(block0_values)
#     print(block0_values_arr)

In [3]:
df_train_input = pd.DataFrame(columns=axis0_arr, data=block0_values_arr,index=axis1_arr)
# print(df_train_input.head(101))
# print(list())

In [4]:
filename = "train_cite_targets.h5"

with h5py.File(filename,'r') as hdf:
    ls =list(hdf.keys())
#     print("list of datasets in this file : \n",ls)
#     print("list of keys in dataset ",hdf.get('train_cite_targets').keys())
#     print(hdf['train_cite_targets']['axis0'])
    
    axis0 = hdf.get('train_cite_targets').get('axis0')
    axis0_arr = np.array(axis0)
    axis0_arr = np.char.decode(axis0_arr) 
#     print(axis0_arr) 
    
#     print(hdf['train_cite_targets']['axis1'])
    axis1 = hdf.get('train_cite_targets').get('axis1')
    axis1_arr = np.array(axis1)
    axis1_arr = np.char.decode(axis1_arr)
#     print(axis1_arr)
    
#     print(hdf['train_cite_targets']['block0_items'])
    block0_items = hdf.get('train_cite_targets').get('block0_items')
    block0_items_arr = np.array(block0_items)
#     print(block0_items_arr)
    
    
#     print(hdf['train_cite_targets']['block0_values'])
    block0_values = hdf.get('train_cite_targets').get('block0_values')
    block0_values_arr = np.array(block0_values)
#     print(block0_values_arr)

In [5]:
df_train_target = pd.DataFrame(columns=axis0_arr, data=block0_values_arr,index=axis1_arr)
target_columns = list(df_train_target.columns.values)

In [6]:
print(len(df_train_input), len(df_train_target))

70988 70988


In [7]:
sample_size = 50000
################
####  PREPROCESSING ΑΛΗΘΕΙΑΣ
###############
df_train_input = df_train_input.iloc[:sample_size]
df_train_target = df_train_target.iloc[:sample_size]

#numero uno leaderboard preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler,normalize

# df_norm = StandardScaler().fit_transform(df_train_input.values)
df_norm = pd.DataFrame(normalize(df_train_input,norm='l2',axis=1),index=df_train_input.index, columns=df_train_input.columns)
df_median = df_norm.sub(df_norm.median(axis=0), axis=1)

tsvd = TruncatedSVD(n_components=128)
df_tsvd = pd.DataFrame(tsvd.fit_transform(df_median), index=df_median.index)
print("finished preprocessing")
df_train_input = df_tsvd
#order by index (cell_id)
df_train_input.sort_index(axis=0,inplace=True)
df_train_target.sort_index(axis=0,inplace=True)


df_merged = pd.merge(df_train_input,df_train_target,left_index=True, right_index=True)
df_merged.head(100)


target_columns = df_train_target.columns.values
# print(target_columns)
# Separate the input and target columns
X = df_merged.drop(target_columns, axis=1)
y = df_merged[target_columns]

finished preprocessing


In [10]:
print("starting gridsearch for MLP Regressor...")
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats  import pearsonr

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#param optimization
param_grid = {
    'hidden_layer_sizes': [(50,),(100,),(50,50), (100,100), (64, 128)],
    'activation':['relu','logistic','tanh'],
    'solver':['adam','lbfgs','sgd'],
    'max_iter':[50,100,200,300,500],
    'alpha':[0.1, 0.2, 0.25],
}
# Initialize the MLPRegressor
# mlp_model = MLPRegressor(hidden_layer_sizes=(64, 128),activation='relu', solver='adam', learning_rate='adaptive', max_iter=500,
#                         alpha=0.01, early_stopping=True,validation_fraction=0.2)
mlp_model  = MLPRegressor()
grid_search = GridSearchCV(mlp_model, param_grid, cv = 5,n_jobs=-1)
# Train the models on the dataset
grid_search.fit(X_train, y_train)

print("best params : " , grid_search.best_params_)
print("best r2 score : ", grid_search.best_score_)

starting MLP Regressor...
best params :  {'activation': 'tanh', 'alpha': 0.2, 'hidden_layer_sizes': (100, 100), 'max_iter': 500, 'solver': 'lbfgs'}
best r2 score :  0.1276774082288487


c:\python3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [9]:
print("starting MLP Regressor...")
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats  import pearsonr

# Initialize the MLPRegressor with best params
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlp_model = MLPRegressor(hidden_layer_sizes=(100,100),activation='tanh', solver='lbfgs', max_iter=1000,
                        alpha=0.2)

# {'activation': 'tanh', 'alpha': 0.2, 'hidden_layer_sizes': (100, 100), 'max_iter': 500, 'solver': 'lbfgs'}
# best r2 score :  0.1276774082288487

# Train the models on the dataset
mlp_model.fit(X_train, y_train)

# Predict the target values using the trained models
mlp_y_preds = mlp_model.predict(X_test)

#get model accuracy
# accuracy = accuracy_score(y_test, mlp_y_preds)
mae = mean_absolute_error(y_test, mlp_y_preds)
mse = mean_squared_error(y_test, mlp_y_preds)
# print("accuracy: ", accuracy)
print("mae: ", mae)
print("mse: ", mse)

#accuracy for multiple columns

threshold = 0.1
corr_loss = []
# print(y_test.shape)
# print(type(y_test))
# print(y_test.iloc[:10, 1])
# print(type(mlp_y_preds))
# print(mlp_y_preds.shape)
# print(mlp_y_preds[:10,1])
for i in range(y_test.shape[1]):  
    correlation_loss_i = 1 - pearsonr(y_test.iloc[:, i], mlp_y_preds[:,i])[0]
    corr_loss.append(correlation_loss_i)
print("Mean Correlation Loss: ", np.mean(corr_loss))
# print("successfull preds: ", np.mean(percentage_list))

starting MLP Regressor...


c:\python3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


mae:  1.026996666152159
mse:  2.446220031372783
Mean Correlation Loss:  0.5743312840233705


In [21]:
## catboost param searching

print("starting gridsearch for CatBoost Regressor...")
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import GridSearchCV,train_test_split

#declare data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Load data into a Pool object
train_data = Pool(X_train, y_train)

# Define the hyperparameter grid to search over
param_grid = {
    'iterations': [100, 500, 1000],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [32, 64, 128]
}

# Create a CatBoost regressor
regressor = CatBoostRegressor(*params)

# Perform grid search using cross-validation
grid_search = GridSearchCV(regressor, param_grid=param_grid, cv=4, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(train_data)

# Print the best parameters and accuracy score
print("Best parameters for catboost: ", grid_search.best_params_)
print("with best negative mean squared error: ", grid_search.best_score_)


starting gridsearch for CatBoost Regressor...


TypeError: Singleton array array(<catboost.core.Pool object at 0x000002D22E944160>, dtype=object) cannot be considered a valid collection.

In [8]:
## run catboost with best params
from sklearn.model_selection import train_test_split

# Create a CatBoost regressor with the best parameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = CatBoostRegressor(iterations=1000, depth=8, learning_rate=0.1, l2_leaf_reg=5, border_count=128)

# Train the model on the training data
regressor.fit(X_train, y_train)

# Predict on the test data
y_pred = regressor.predict(X_test)

# accuracy = accuracy_score(y_test, mlp_y_preds)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# print("accuracy: ", accuracy)
print("mae: ", mae)
print("mse: ", mse)

#accuracy for multiple columns

threshold = 0.1
corr_loss = []
# print(y_test.shape)
# print(type(y_test))
# print(y_test.iloc[:10, 1])
# print(type(mlp_y_preds))
# print(mlp_y_preds.shape)
# print(mlp_y_preds[:10,1])
for i in range(y_test.shape[1]):  
    correlation_loss_i = 1 - pearsonr(y_test.iloc[:, i], y_pred[:,i])[0]
    corr_loss.append(correlation_loss_i)
print("Mean Correlation Loss: ", np.mean(corr_loss))

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/data_providers.cpp:612: Currently only multi-regression, multilabel and survival objectives work with multidimensional target

In [22]:
from catboost import CatBoostRegressor, Pool

# Load data into a Pool object
train_data = Pool(X_train, y_train)

# Define the model parameters
params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'depth': 6,
    'loss_function': 'MultiRMSE'
}

# Create a CatBoost regressor
multi_regressor = CatBoostRegressor(**params)

# Train the model on the training data
multi_regressor.fit(train_data)

# Predict on the test data
test_data = Pool(X_test)
y_pred = multi_regressor.predict(test_data)


0:	learn: 21.8376473	total: 3.99s	remaining: 1h 6m 29s
1:	learn: 21.3805771	total: 7.56s	remaining: 1h 2m 51s
2:	learn: 20.9911736	total: 11.2s	remaining: 1h 2m 13s
3:	learn: 20.6211939	total: 14.9s	remaining: 1h 2m
4:	learn: 20.2827165	total: 18.6s	remaining: 1h 1m 42s
5:	learn: 20.0445764	total: 22.3s	remaining: 1h 1m 26s
6:	learn: 19.8112624	total: 25.9s	remaining: 1h 1m 18s
7:	learn: 19.5672409	total: 29.6s	remaining: 1h 1m 8s
8:	learn: 19.3560135	total: 33.3s	remaining: 1h 1m 11s
9:	learn: 19.1688421	total: 37s	remaining: 1h 1m 4s
10:	learn: 19.0145480	total: 40.7s	remaining: 1h 59s
11:	learn: 18.8573291	total: 44.4s	remaining: 1h 51s
12:	learn: 18.7008914	total: 48s	remaining: 1h 45s
13:	learn: 18.5942065	total: 51.7s	remaining: 1h 39s
14:	learn: 18.4883974	total: 55.3s	remaining: 1h 30s
15:	learn: 18.3988180	total: 58.9s	remaining: 1h 23s
16:	learn: 18.3102593	total: 1m 2s	remaining: 1h 32s
17:	learn: 18.2464861	total: 1m 6s	remaining: 1h 31s
18:	learn: 18.1752126	total: 1m 10s	

150:	learn: 15.5654539	total: 9m 19s	remaining: 52m 26s
151:	learn: 15.5579388	total: 9m 23s	remaining: 52m 23s
152:	learn: 15.5471315	total: 9m 27s	remaining: 52m 21s
153:	learn: 15.5360727	total: 9m 31s	remaining: 52m 18s
154:	learn: 15.5260925	total: 9m 35s	remaining: 52m 14s
155:	learn: 15.5165562	total: 9m 38s	remaining: 52m 11s
156:	learn: 15.5093487	total: 9m 42s	remaining: 52m 8s
157:	learn: 15.5036040	total: 9m 46s	remaining: 52m 4s
158:	learn: 15.4964520	total: 9m 49s	remaining: 52m
159:	learn: 15.4912450	total: 9m 53s	remaining: 51m 55s
160:	learn: 15.4831595	total: 9m 57s	remaining: 51m 51s
161:	learn: 15.4709713	total: 10m	remaining: 51m 47s
162:	learn: 15.4592319	total: 10m 4s	remaining: 51m 43s
163:	learn: 15.4530417	total: 10m 8s	remaining: 51m 39s
164:	learn: 15.4421962	total: 10m 11s	remaining: 51m 35s
165:	learn: 15.4337887	total: 10m 15s	remaining: 51m 31s
166:	learn: 15.4240965	total: 10m 19s	remaining: 51m 28s
167:	learn: 15.4161948	total: 10m 22s	remaining: 51m 2

295:	learn: 14.4977063	total: 18m 25s	remaining: 43m 50s
296:	learn: 14.4907194	total: 18m 29s	remaining: 43m 46s
297:	learn: 14.4861064	total: 18m 33s	remaining: 43m 42s
298:	learn: 14.4787839	total: 18m 37s	remaining: 43m 38s
299:	learn: 14.4734660	total: 18m 40s	remaining: 43m 35s
300:	learn: 14.4623839	total: 18m 44s	remaining: 43m 31s
301:	learn: 14.4591753	total: 18m 48s	remaining: 43m 27s
302:	learn: 14.4514251	total: 18m 52s	remaining: 43m 24s
303:	learn: 14.4449683	total: 18m 55s	remaining: 43m 20s
304:	learn: 14.4415502	total: 18m 59s	remaining: 43m 17s
305:	learn: 14.4332097	total: 19m 3s	remaining: 43m 13s
306:	learn: 14.4265674	total: 19m 7s	remaining: 43m 9s
307:	learn: 14.4200405	total: 19m 10s	remaining: 43m 5s
308:	learn: 14.4128040	total: 19m 14s	remaining: 43m 2s
309:	learn: 14.4071874	total: 19m 18s	remaining: 42m 58s
310:	learn: 14.4016994	total: 19m 21s	remaining: 42m 54s
311:	learn: 14.3926597	total: 19m 25s	remaining: 42m 50s
312:	learn: 14.3861968	total: 19m 29

440:	learn: 13.6058611	total: 27m 17s	remaining: 34m 35s
441:	learn: 13.6024358	total: 27m 20s	remaining: 34m 31s
442:	learn: 13.5971084	total: 27m 24s	remaining: 34m 27s
443:	learn: 13.5897592	total: 27m 28s	remaining: 34m 23s
444:	learn: 13.5820452	total: 27m 31s	remaining: 34m 20s
445:	learn: 13.5739707	total: 27m 35s	remaining: 34m 16s
446:	learn: 13.5683500	total: 27m 39s	remaining: 34m 13s
447:	learn: 13.5644659	total: 27m 43s	remaining: 34m 9s
448:	learn: 13.5587439	total: 27m 46s	remaining: 34m 5s
449:	learn: 13.5551565	total: 27m 50s	remaining: 34m 1s
450:	learn: 13.5472269	total: 27m 54s	remaining: 33m 58s
451:	learn: 13.5412755	total: 27m 57s	remaining: 33m 54s
452:	learn: 13.5358723	total: 28m 1s	remaining: 33m 50s
453:	learn: 13.5285514	total: 28m 5s	remaining: 33m 46s
454:	learn: 13.5253281	total: 28m 8s	remaining: 33m 42s
455:	learn: 13.5194254	total: 28m 12s	remaining: 33m 39s
456:	learn: 13.5131228	total: 28m 16s	remaining: 33m 35s
457:	learn: 13.5064142	total: 28m 19s

585:	learn: 12.7835195	total: 36m 9s	remaining: 25m 32s
586:	learn: 12.7796032	total: 36m 12s	remaining: 25m 28s
587:	learn: 12.7733943	total: 36m 16s	remaining: 25m 25s
588:	learn: 12.7675911	total: 36m 20s	remaining: 25m 21s
589:	learn: 12.7612911	total: 36m 24s	remaining: 25m 17s
590:	learn: 12.7543684	total: 36m 27s	remaining: 25m 14s
591:	learn: 12.7463769	total: 36m 31s	remaining: 25m 10s
592:	learn: 12.7399981	total: 36m 35s	remaining: 25m 6s
593:	learn: 12.7326868	total: 36m 38s	remaining: 25m 3s
594:	learn: 12.7255853	total: 36m 42s	remaining: 24m 59s
595:	learn: 12.7215187	total: 36m 46s	remaining: 24m 55s
596:	learn: 12.7146877	total: 36m 50s	remaining: 24m 52s
597:	learn: 12.7115343	total: 36m 54s	remaining: 24m 48s
598:	learn: 12.7048604	total: 36m 58s	remaining: 24m 45s
599:	learn: 12.6954432	total: 37m 2s	remaining: 24m 41s
600:	learn: 12.6868579	total: 37m 6s	remaining: 24m 38s
601:	learn: 12.6805752	total: 37m 10s	remaining: 24m 34s
602:	learn: 12.6754141	total: 37m 13

730:	learn: 12.0304143	total: 45m 5s	remaining: 16m 35s
731:	learn: 12.0239337	total: 45m 9s	remaining: 16m 32s
732:	learn: 12.0167930	total: 45m 13s	remaining: 16m 28s
733:	learn: 12.0119768	total: 45m 16s	remaining: 16m 24s
734:	learn: 12.0050569	total: 45m 20s	remaining: 16m 20s
735:	learn: 12.0025165	total: 45m 24s	remaining: 16m 17s
736:	learn: 11.9936276	total: 45m 27s	remaining: 16m 13s
737:	learn: 11.9880381	total: 45m 31s	remaining: 16m 9s
738:	learn: 11.9842453	total: 45m 35s	remaining: 16m 6s
739:	learn: 11.9791067	total: 45m 38s	remaining: 16m 2s
740:	learn: 11.9744595	total: 45m 42s	remaining: 15m 58s
741:	learn: 11.9675259	total: 45m 46s	remaining: 15m 54s
742:	learn: 11.9617911	total: 45m 49s	remaining: 15m 51s
743:	learn: 11.9558033	total: 45m 53s	remaining: 15m 47s
744:	learn: 11.9521999	total: 45m 57s	remaining: 15m 43s
745:	learn: 11.9468197	total: 46m	remaining: 15m 39s
746:	learn: 11.9430473	total: 46m 4s	remaining: 15m 36s
747:	learn: 11.9395957	total: 46m 7s	rema

876:	learn: 11.3152596	total: 54m	remaining: 7m 34s
877:	learn: 11.3128553	total: 54m 4s	remaining: 7m 30s
878:	learn: 11.3066828	total: 54m 8s	remaining: 7m 27s
879:	learn: 11.3041465	total: 54m 11s	remaining: 7m 23s
880:	learn: 11.2998677	total: 54m 15s	remaining: 7m 19s
881:	learn: 11.2932036	total: 54m 19s	remaining: 7m 16s
882:	learn: 11.2910163	total: 54m 22s	remaining: 7m 12s
883:	learn: 11.2879324	total: 54m 26s	remaining: 7m 8s
884:	learn: 11.2842683	total: 54m 29s	remaining: 7m 4s
885:	learn: 11.2792487	total: 54m 33s	remaining: 7m 1s
886:	learn: 11.2725564	total: 54m 37s	remaining: 6m 57s
887:	learn: 11.2667709	total: 54m 41s	remaining: 6m 53s
888:	learn: 11.2626219	total: 54m 44s	remaining: 6m 50s
889:	learn: 11.2565498	total: 54m 48s	remaining: 6m 46s
890:	learn: 11.2508487	total: 54m 52s	remaining: 6m 42s
891:	learn: 11.2431884	total: 54m 55s	remaining: 6m 39s
892:	learn: 11.2387651	total: 54m 59s	remaining: 6m 35s
893:	learn: 11.2343789	total: 55m 3s	remaining: 6m 31s
89

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats  import pearsonr

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# print("accuracy: ", accuracy)
print("mae: ", mae)
print("mse: ", mse)

#accuracy for multiple columns

threshold = 0.1
corr_loss = []
# print(y_test.shape)
# print(type(y_test))
# print(y_test.iloc[:10, 1])
# print(type(mlp_y_preds))
# print(mlp_y_preds.shape)
# print(mlp_y_preds[:10,1])
for i in range(y_test.shape[1]):  
    correlation_loss_i = 1 - pearsonr(y_test.iloc[:, i], y_pred[:,i])[0]
    corr_loss.append(correlation_loss_i)
print("Mean Correlation Loss: ", np.mean(corr_loss))

mae:  0.9392186204390125
mse:  2.085699547837247
Mean Correlation Loss:  0.6972741867872163


In [8]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



# Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a MLPRegressor model
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100,100),activation='tanh', solver='lbfgs', max_iter=1000,
                        alpha=0.2)

# Train the MLPRegressor on the training data
mlp_regressor.fit(X_train, y_train)

# Use the MLPRegressor to predict on the test data
y_pred_mlp = mlp_regressor.predict(X_test)

# Build a CatBoostRegressor model
catboost_regressor = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='MultiRMSE', random_state=42)
catboost_data = Pool(X_train, y_train)
# Train the CatBoostRegressor on the training data
catboost_regressor.fit(catboost_data)

# Use the CatBoostRegressor to predict on the test data
catboost_X_test = Pool(X_test)
y_pred_catboost = catboost_regressor.predict(X_test)

# Concatenate the predicted outputs from both models
concatenated_preds = np.concatenate((y_pred_mlp, y_pred_catboost), axis=1)

# Build a new MLPRegressor model to combine the predictions
combined_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)

# Train the combined MLPRegressor model on the concatenated predictions
combined_regressor.fit(concatenated_preds, y_test)

# Use the MLPRegressor to predict on the test data
y_pred_combined = combined_regressor.predict(concatenated_preds)

# Calculate the mean squared error for the combined MLPRegressor
mse_combined = mean_squared_error(y_test, y_pred_combined)


NameError: name 'X_train' is not defined

In [28]:
print(mse_combined)

1.7581193192038813


In [9]:
#calculate and divide by nonzero median
row_median = lambda row: np.median(row[row != 0])
nonzero_medians = df_train_input.apply(row_median,axis=1) 
df_div= df_train_input.div(nonzero_medians,axis=0) 
df_div.head(10)

,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,ENSG00000153975_ZUP1,ENSG00000086827_ZW10,ENSG00000174442_ZWILCH,ENSG00000122952_ZWINT,ENSG00000198205_ZXDA,ENSG00000198455_ZXDB,ENSG00000070476_ZXDC,ENSG00000162378_ZYG11B,ENSG00000159840_ZYX,ENSG00000074755_ZZEF1
000141b198e6,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,...,0.0,0.0,0.000000,1.271958,0.0,0.000000,0.00000,0.00000,0.000000,0.0
0003fcc59086,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.842476,...,0.0,1.0,0.842476,1.252469,0.0,0.000000,0.00000,0.00000,0.000000,0.0
000441bba05a,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.0
0004897a2b2b,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,1.000000,0.0
000517abfb58,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,...,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.00000,1.00000,1.000000,0.0
00056fb385f2,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,1.00000,0.00000,1.000000,0.0
0005b45cd15b,0.0,0.0,0.0,0.0,0.0,0.0,1.104063,1.000000,0.0,0.000000,...,0.0,0.0,1.000000,1.358000,0.0,0.000000,0.82448,0.82448,1.000000,0.0
00097bbec9a2,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.838487,...,0.0,1.0,0.000000,0.838487,0.0,0.838487,0.00000,0.00000,1.000000,0.0
0009a974f266,0.0,0.0,0.0,0.0,0.0,0.0,0.838241,0.838241,0.0,0.000000,...,1.0,1.0,0.838241,1.095511,0.0,0.000000,0.00000,0.00000,0.838241,0.0
000a9f1e0b14,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1.0,0.0,0.000000,1.266184,0.0,0.000000,0.00000,0.00000,0.000000,0.0


In [10]:
# transform with log1p
df_log = df_div.applymap(np.log1p)
df_log.head(10)

,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,ENSG00000153975_ZUP1,ENSG00000086827_ZW10,ENSG00000174442_ZWILCH,ENSG00000122952_ZWINT,ENSG00000198205_ZXDA,ENSG00000198455_ZXDB,ENSG00000070476_ZXDC,ENSG00000162378_ZYG11B,ENSG00000159840_ZYX,ENSG00000074755_ZZEF1
000141b198e6,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.693147,0.0,0.000000,...,0.000000,0.000000,0.000000,0.820642,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0003fcc59086,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.611110,...,0.000000,0.693147,0.611110,0.812027,0.0,0.000000,0.000000,0.000000,0.000000,0.0
000441bba05a,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.693147,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0004897a2b2b,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.693147,0.0
000517abfb58,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.693147,0.0,0.000000,...,0.000000,0.000000,0.693147,0.000000,0.0,0.000000,0.000000,0.693147,0.693147,0.0
00056fb385f2,0.0,0.000000,0.0,0.0,0.0,0.0,0.693147,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.693147,0.000000,0.693147,0.0
0005b45cd15b,0.0,0.000000,0.0,0.0,0.0,0.0,0.743870,0.693147,0.0,0.000000,...,0.000000,0.000000,0.693147,0.857814,0.0,0.000000,0.601295,0.601295,0.693147,0.0
00097bbec9a2,0.0,0.000000,0.0,0.0,0.0,0.0,0.693147,0.000000,0.0,0.608943,...,0.000000,0.693147,0.000000,0.608943,0.0,0.608943,0.000000,0.000000,0.693147,0.0
0009a974f266,0.0,0.000000,0.0,0.0,0.0,0.0,0.608809,0.608809,0.0,0.000000,...,0.693147,0.693147,0.608809,0.739797,0.0,0.000000,0.000000,0.000000,0.608809,0.0
000a9f1e0b14,0.0,0.693147,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.693147,0.000000,0.000000,0.818097,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [11]:
df_train_input = df_log

In [12]:
# inner join
# df_merged = df_train_input.merge(df_train_target,how='inner')#,lsuffix='l_',rsuffix='r_')
df_merged = pd.merge(df_train_input,df_train_target,left_index=True, right_index=True)
df_merged.head(100)

,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
000141b198e6,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.693147,0.0,0.00000,...,0.144601,10.489143,0.939881,0.841505,3.309048,1.978804,13.027639,-0.208923,2.542908,8.071543
0003fcc59086,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.61111,...,-0.595230,6.023785,0.408263,-0.497212,-0.133765,2.195598,9.705816,-0.580132,7.645536,5.570200
000441bba05a,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,0.814176,9.285316,0.299309,1.803804,0.051648,1.212216,3.949074,1.440157,3.350088,0.584580
0004897a2b2b,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,4.829338,5.624609,2.015439,4.492601,-1.151961,3.715674,5.097805,1.668119,1.643337,5.632807
000517abfb58,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.693147,0.0,0.00000,...,-1.066876,7.712044,-0.332271,-0.608914,2.938849,0.370199,7.530865,-0.520287,2.078689,2.115581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0054b21867c8,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,0.473038,2.826349,0.116303,0.640916,-0.020092,-0.048039,3.671904,1.891259,3.929412,2.359597
005569028ddd,0.0,0.0,0.0,0.693147,0.0,0.0,0.693147,0.000000,0.0,0.00000,...,-0.148417,10.783627,1.109807,0.961539,0.070386,1.774466,4.031822,0.486662,3.392267,5.753120
005633f1ea16,0.0,0.0,0.0,0.000000,0.0,0.0,0.611884,0.000000,0.0,0.00000,...,0.593003,6.540831,2.188413,0.337904,-0.449624,-0.181593,5.820107,-0.139723,5.907623,2.739270
0057cff787fe,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,-0.366140,0.343975,0.556323,-1.113267,0.377743,-0.448942,0.539068,0.497742,2.360866,0.351199


In [13]:
#calculate corr coef
target_columns = "','".join(list(axis0_arr))

# corr_df = df_merged.corr()[target_columns]
# corr_threshold = .42
# hcr_df = df_merged.loc[(corr_df.abs() > corr_threshold)]

In [14]:
df_merged.head()
len(df_merged)

3000

In [17]:
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor

target_columns = df_train_target.columns.values
print(target_columns)
# Separate the input and target columns
X = df_merged.drop(target_columns, axis=1)
y = df_merged[target_columns]

# Initialize the MLPRegressor and CatBoostRegressor models
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500)
# catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1)

# Train the models on the dataset
mlp_model.fit(X, y)
# catboost_model.fit(X, y)

# Predict the target values using the trained models
mlp_preds = mlp_model.predict(X)
# catboost_preds = catboost_model.predict(X)

# mlp1_score = mlp_model.score(test_features, test_target1)
# mlp2_score = mlp2.score(test_features, test_target2)

print(mlp_preds[:100])

['CD86' 'CD274' 'CD270' 'CD155' 'CD112' 'CD47' 'CD48' 'CD40' 'CD154'
 'CD52' 'CD3' 'CD8' 'CD56' 'CD19' 'CD33' 'CD11c' 'HLA-A-B-C' 'CD45RA'
 'CD123' 'CD7' 'CD105' 'CD49f' 'CD194' 'CD4' 'CD44' 'CD14' 'CD16' 'CD25'
 'CD45RO' 'CD279' 'TIGIT' 'Mouse-IgG1' 'Mouse-IgG2a' 'Mouse-IgG2b'
 'Rat-IgG2b' 'CD20' 'CD335' 'CD31' 'Podoplanin' 'CD146' 'IgM' 'CD5'
 'CD195' 'CD32' 'CD196' 'CD185' 'CD103' 'CD69' 'CD62L' 'CD161' 'CD152'
 'CD223' 'KLRG1' 'CD27' 'CD107a' 'CD95' 'CD134' 'HLA-DR' 'CD1c' 'CD11b'
 'CD64' 'CD141' 'CD1d' 'CD314' 'CD35' 'CD57' 'CD272' 'CD278' 'CD58' 'CD39'
 'CX3CR1' 'CD24' 'CD21' 'CD11a' 'CD79b' 'CD244' 'CD169' 'integrinB7'
 'CD268' 'CD42b' 'CD54' 'CD62P' 'CD119' 'TCR' 'Rat-IgG1' 'Rat-IgG2a'
 'CD192' 'CD122' 'FceRIa' 'CD41' 'CD137' 'CD163' 'CD83' 'CD124' 'CD13'
 'CD2' 'CD226' 'CD29' 'CD303' 'CD49b' 'CD81' 'IgD' 'CD18' 'CD28' 'CD38'
 'CD127' 'CD45' 'CD22' 'CD71' 'CD26' 'CD115' 'CD63' 'CD304' 'CD36'
 'CD172a' 'CD72' 'CD158' 'CD93' 'CD49a' 'CD49d' 'CD73' 'CD9' 'TCRVa7.2'
 'TCRVd2' 'LOX-

In [ ]:
#numero uno leaderboard preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

df_norm = StandardScaler().fit_transform(df_train_input.values)
df_median = np.median(df_norm, axis=0)
df_norm_median = df_norm - df_median

tsvd = TruncatedSVD(n_components=128)
df_tsvd = tsvd.fit_transform(df_norm_median)

df

